# Welcome to the Lab 🥼🧪

### Precision Address Research

In this notebook, we'll demonstrate how to research specific addresses using the Parcl Labs API. Whether you need to analyze a single property or a curated list, our flexible matching system ensures precise identification and complete event histories.

**You'll learn how to:**
* Match addresses exactly (even with formatting variations)
* Access complete property timelines
* Search individual or lists of addresses with equal precision

**Need help getting started?**
* Get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.
* To run this immediately, you can use Google Colab. Remember, you must set your PARCL_LABS_API_KEY.
* Property level data is a premium feature and you will need a paid account to access it. You can get upgrade instantly [here](https://dashboard.parcllabs.com/login).

Run in --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/getting_started/address-search-notebook.ipynb)

# Setup 🔧
Basic setup requires three steps:

* Install Parcl Labs SDK and required packages
* Import dependencies
* Initialize client (remember to set your PARCL_LABS_API_KEY)

Run each cell below to get started.

In [ ]:
%pip install --upgrade parcllabs

In [17]:
import os
import time
from datetime import datetime, timedelta
import pandas as pd
from parcllabs import ParclLabsClient

In [18]:
# Instantiate the client
client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"),
    limit=10  # set default limit
)

In [19]:
# Define your local folder to download the data
download_dir = '../../outputs' # Replace with your own folder

# Individual Address Research
Quick workflow for single-address property research:

🔍 Search

* Input any address format
* Returns unique Parcl Property ID
* Handles variations and misspellings

📋 Events

* Pull complete property timeline
* All events (sales, listings, rentals)
* Optional date and event type filters

💾 Export

* Auto-named CSV with full history
* Ready for analysis

In [ ]:
# Example of searching a single address
# Note: The system is flexible and can handle:
#   - Misspellings (e.g., 'waterfal' instead of 'waterfall')
#   - Different city formats (e.g., 'springhill' or 'spring hill')
#   - Various street suffixes (e.g., 'dr', 'drive', 'drv')
#   - Inconsistent spacing and capitalization

# Input address
input_address = {
    "address": "2535 waterfal dr",
    "city": "springhill",
    "state_abbreviation": "FL",
    "zip_code": "34608"
}

# Perform the address search
result = client.property_address.search.retrieve(
    addresses=[input_address]
)

# Store the parcl_property_id for future use
matched_property_id = None

# Check if we found a match
if not result.empty and pd.notna(result['parcl_property_id'].iloc[0]):
    print("✓ Successfully matched address!")

    # Store the matched ID
    matched_property_id = result['parcl_property_id'].iloc[0]

    # Create a readable dataframe combining input and result
    summary_df = pd.DataFrame({
        'Input Address': [f"{input_address['address']}, {input_address['city']}, {input_address['state_abbreviation']} {input_address['zip_code']}"],
        'Parcl Property ID': [matched_property_id],
    })

    # Display the formatted results
    print("\nAddress Matching Results:")
    print(summary_df.to_string(index=False))
else:
    print("No match found for the provided address.")

In [ ]:
# Now retrieve the property events if we found a match
if matched_property_id:
    # Event types available:
    # - ALL: Returns all event types
    # - SALE: Returns only sale events
    # - RENTAL: Returns only rental events
    # - LISTING: Returns only listing events

    # Note: You can customize the search by:
    # - Filtering by date range using start_date and end_date (format: YYYY-MM-DD)
    # - Choosing specific event types

    # Example: Get all events (you can modify these parameters as needed)
    property_events = client.property.events.retrieve(
        parcl_property_ids=[matched_property_id],
        event_type='ALL'
        # Uncomment and modify these lines to customize your search:
        # event_type='SALE',  # Options: 'ALL', 'SALE', 'RENTAL', 'LISTING'
        # start_date='2020-01-01',  # Format: YYYY-MM-DD
        # end_date='2024-12-31'    # Format: YYYY-MM-DD
    )

    # Sort events by date (most recent first)
    property_events_sorted = property_events.sort_values(by='event_date', ascending=False)
    # Count event types
    event_counts = property_events['event_type'].value_counts()

    # Print summary
    print("\nEvent History Summary:")
    print(f"Total events found: {len(property_events)}")
    for event_type, count in event_counts.items():
        print(f"- {event_type}: {count} events")

    # Display the events dataframe
    print("\nDetailed Event History (Most Recent First):")
    print(property_events_sorted)

    # Store the results for potential CSV export in next step
    latest_property_events = property_events_sorted


In [ ]:
# Export the event history to CSV with a descriptive filename
try:
    if not latest_property_events.empty:
        # Create a clean address string for the filename
        clean_address = input_address['address'].replace(' ', '_').lower()
        clean_city = input_address['city'].replace(' ', '_').lower()

        # Create descriptive filename with address, event types, and date
        filename = f"{clean_address}_{clean_city}_{input_address['state_abbreviation']}_{input_address['zip_code']}_event_history_{datetime.now().strftime('%Y%m%d')}.csv"

        # Create full file path in the downloads directory
        file_path = os.path.join(download_dir, filename)

        # Export to CSV
        latest_property_events.to_csv(file_path, index=False)

        print(f"\nEvent history exported successfully!")
        print(f"File saved as: {filename}")
        print(f"Full path: {file_path}")
    else:
        print("\nNo event history available to export.")
except NameError:
    print("\nNo event history available to export.")


# Batch Property Research
Quick workflow for researching multiple addresses at once:

🔍 Batch Search

* Copy/paste multiple addresses
* Flexible format handling for each
* Returns matched Property IDs
* Shows success rate

📋 Bulk Events

* Pull timeline for all properties
* Preview results before export
* Filter by date or event type

💾 Export

* Single CSV with all histories
* Includes original addresses
* Ready for analysis

In [23]:
# Multiple Address Search Example
# To add more addresses, copy and paste any block between the { } symbols
# and add a comma after the closing }
# Make sure to fill in: address, city, state_abbreviation, and zip_code

input_addresses = [
    {
        "address": "2535 waterfall drive",
        "city": "spring hill",
        "state_abbreviation": "FL",
        "zip_code": "34608"
    },
    {
        # Add your second address here
        "address": "5603 Woodland Dr",
        "city": "Douglasville",
        "state_abbreviation": "GA",
        "zip_code": "30135"
    },
    {
        # Add your third address here
        "address": "23836 N 58th Ave",
        "city": "Glendale",
        "state_abbreviation": "AZ",
        "zip_code": "85310"
    }
]

# Remove any empty address placeholders
input_addresses = [addr for addr in input_addresses if addr["address"] != ""]


In [ ]:
# Perform the address search
results = client.property_address.search.retrieve(
    addresses=input_addresses
)

# Count valid matches (non-NaN parcl_property_id)
valid_matches = results['parcl_property_id'].notna().sum()

# Check matches and create summary
if not results.empty:
    print(f"✓ Successfully matched {valid_matches} out of {len(input_addresses)} addresses!")

    # Create readable dataframe linking inputs to property IDs
    summary_data = []
    for i, addr in enumerate(input_addresses):
        parcl_id = results['parcl_property_id'].iloc[i] if i < len(results) else None
        summary_data.append({
            'Input Address': f"{addr['address']}, {addr['city']}, {addr['state_abbreviation']} {addr['zip_code']}",
            'Parcl Property ID': int(parcl_id) if pd.notna(parcl_id) else 'No match'
        })

    summary_df = pd.DataFrame(summary_data)

    # Display the formatted results
    print("\nAddress Matching Results:")
    print(summary_df.to_string(index=False))

    # Store only the valid property IDs for future use
    matched_property_ids = results[results['parcl_property_id'].notna()]['parcl_property_id'].tolist()
else:
    print("No matches found for the provided addresses.")

In [ ]:
# Retrieve event history for all matched properties
if matched_property_ids:
    # Event types available:
    # - ALL: Returns all event types
    # - SALE: Returns only sale events
    # - RENTAL: Returns only rental events
    # - LISTING: Returns only listing events

    # You can customize the search by uncommenting and modifying these parameters:
    property_events = client.property.events.retrieve(
        parcl_property_ids=matched_property_ids,
        event_type='ALL'
        # event_type='SALE',          # Options: 'ALL', 'SALE', 'RENTAL', 'LISTING'
        # start_date='2020-01-01',    # Format: YYYY-MM-DD
        # end_date='2024-12-31'       # Format: YYYY-MM-DD
    )

    # Sort by date (most recent first) and add address information
    if not property_events.empty:
        # Create address lookup dictionary
        address_lookup = {row['Parcl Property ID']: row['Input Address']
                         for _, row in summary_df.iterrows()}

        # Add address information to events
        property_events['input_address'] = property_events['parcl_property_id'].map(address_lookup)

        # Sort by date
        property_events_sorted = property_events.sort_values(by=['parcl_property_id', 'event_date'],
                                                           ascending=[True, False])

        # Calculate event counts
        total_events = len(property_events)
        events_by_type = property_events['event_type'].value_counts()
        events_by_property = property_events.groupby('parcl_property_id').size()

        # Print summary
        print("\n✓ Successfully retrieved event histories!")
        print(f"\nSummary:")
        print(f"- Total properties searched: {len(matched_property_ids)}")
        print(f"- Total events found: {total_events}")
        print("\nEvents by type:")
        for event_type, count in events_by_type.items():
            print(f"- {event_type}: {count}")

        # Preview results (first 2 events for first 2 properties)
        print("\nPreview of results:")
        previewed_properties = 0
        for property_id in matched_property_ids:
            if previewed_properties >= 2:
                break
            property_events = property_events_sorted[
                property_events_sorted['parcl_property_id'] == property_id
            ].head(2)
            if not property_events.empty:
                print(f"\nEvents for {address_lookup[property_id]}:")
                print(property_events[['event_type', 'event_date']].to_string())
                previewed_properties += 1

        # Store results for CSV export
        latest_property_events = property_events_sorted
        print("\nFull results ready for CSV export in next step...")

    else:
        print("No events found for the specified parameters.")
else:
    print("No valid property IDs to search.")

In [ ]:
# Export the event histories to CSV
if 'latest_property_events' in locals() and not latest_property_events.empty:
    # Create filename with property count and date
    property_count = len(matched_property_ids)
    filename = f"property_events_{property_count}_properties_{datetime.now().strftime('%Y%m%d')}.csv"

    # Create full file path in the downloads directory
    file_path = os.path.join(download_dir, filename)

    # Export to CSV
    latest_property_events.to_csv(file_path, index=False)

    print(f"\nEvent histories exported successfully!")
    print(f"File saved as: {filename}")
    print(f"Full path: {file_path}")
    print(f"\nContains event histories for {property_count} properties")
else:
    print("\nNo event histories available to export.")